In [4]:
import pyspark

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField,StructField,StringType,IntegerType
from pyspark.sql.types import ArrayType,DoubleType,BooleanType
from pyspark.sql.functions import *

In [6]:
spark=SparkSession.builder.appName("FCC Assessment").getOrCreate()

In [114]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField,StringType,IntegerType
from pyspark.sql.types import ArrayType,DoubleType,BooleanType
from pyspark.sql.functions  import column
from pyspark.sql import functions as F

In [8]:
#####STEP 1 --LOAD USER TABLE

In [9]:
user_schema = StructType([
    StructField("user_id", StringType(), True),
    StructField("ip_address", StringType(), True),
    StructField("social_media_handle", StringType(), True),
    StructField("email", StringType(), True)
])

In [10]:
user_df = spark.read.csv(
    "C:/Users/lenovo/Downloads/user.csv", schema=user_schema, header=True, sep=",")

In [12]:
#load user_registration

In [13]:
user_registration_df=spark.read.csv(
    "C:/Users/lenovo/Downloads/user_registration.csv", inferSchema=True, header=True, sep=",")

In [ ]:
#load user_play_session

In [14]:
user_play_session_df=spark.read.csv(
    "C:/Users/lenovo/Downloads/user_play_session.csv", inferSchema=True, header=True, sep=",")

In [15]:
#user_play_session_df.show()

In [ ]:
# load user_plan

In [100]:
user_plan_df=spark.read.csv("C:/Users/lenovo/Downloads/user_plan.csv", inferSchema=True, header=True, sep=",")

In [101]:
user_plan_df.show(2)

+--------------------+-----------------+-------+-------------------+-------------------+
|user_registration_id|payment_detail_id|plan_id|         start_date|           end_date|
+--------------------+-----------------+-------+-------------------+-------------------+
|                   1|                1|      2|2024-03-02 01:00:00|9999-01-01 01:00:00|
|                   2|                2|      3|2024-03-31 02:00:00|2024-10-06 02:00:00|
+--------------------+-----------------+-------+-------------------+-------------------+
only showing top 2 rows



In [ ]:
#Load user_payment_detail

In [17]:
user_payment_detail_df=spark.read.csv("C:/Users/lenovo/Downloads/user_payment_detail.csv", inferSchema=True, header=True, sep=",")

In [ ]:
#load status_code

In [18]:
user_status_code_df=spark.read.csv("C:/Users/lenovo/Downloads/status_code.csv", inferSchema=True, header=True, sep=",")

In [ ]:
#load plan_payment_frequency

In [19]:
user_payment_frequency_df=spark.read.csv(
    "C:/Users/lenovo/Downloads/plan_payment_frequency.csv",inferSchema=True, header=True, sep=",")

In [ ]:
# load channel_code

In [20]:
user_channel_code_df=spark.read.csv("C:/Users/lenovo/Downloads/channel_code.csv", inferSchema=True, header=True, sep=",")

In [ ]:
# load plan.csv

In [21]:
user_plan_df=spark.read.csv("C:/Users/lenovo/Downloads/plan.csv", inferSchema=True, header=True, sep=",")

In [22]:
#####STEP 2:  CREATE DIM AND FACT TABLE 

In [23]:
dim_user=user_df.dropDuplicates()

In [2]:
"""dim_user = user_df.join(user_registration_df, "user_id", "left") \
    .select(
        col("user_id"),
        col("ip_address"),
        col("social_media_handle"),
        col("username"),
        col("first_name"),
        col("last_name")
    ).dropDuplicates()"""

'dim_user = user_df.join(user_registration_df, "user_id", "left")     .select(\n        col("user_id"),\n        col("ip_address"),\n        col("social_media_handle"),\n        col("username"),\n        col("first_name"),\n        col("last_name")\n    ).dropDuplicates()'

In [26]:
dim_plan = user_plan_df.select("plan_id", "payment_frequency_code", "cost_amount").dropDuplicates()

In [27]:
dim_plan.show(1)

+-------+----------------------+-----------+
|plan_id|payment_frequency_code|cost_amount|
+-------+----------------------+-----------+
|      1|               MONTHLY|       1.99|
+-------+----------------------+-----------+
only showing top 1 row



In [28]:
dim_payment_frequency = user_payment_frequency_df.dropDuplicates()

In [29]:
dim_payment_frequency.show(1)

+----------------------+-------------------+------------------+
|payment_frequency_code|english_description|french_description|
+----------------------+-------------------+------------------+
|               ONETIME|            Onetime|          Une fois|
+----------------------+-------------------+------------------+
only showing top 1 row



In [30]:
dim_status = user_status_code_df.dropDuplicates()

In [31]:
dim_status.show(1)

+------------------------+-------------------+--------------------+
|play_session_status_code|english_description|  french_description|
+------------------------+-------------------+--------------------+
|                 TIMEOUT|    Session timeout|Délai d'expiratio...|
+------------------------+-------------------+--------------------+
only showing top 1 row



In [32]:
dim_channel = user_channel_code_df.dropDuplicates()

In [33]:
dim_channel.show(1)

+-------------------------+-------------------+------------------+
|play_session_channel_code|english_description|french_description|
+-------------------------+-------------------+------------------+
|                  BROWSER|            Browser|        Navigateur|
+-------------------------+-------------------+------------------+
only showing top 1 row



In [34]:
dim_payment_method = user_payment_detail_df.dropDuplicates()

In [35]:
dim_payment_method.show(1)

+-----------------+--------------------+--------------------+---------------------+
|payment_detail_id| payment_method_code|payment_method_value|payment_method_expiry|
+-----------------+--------------------+--------------------+---------------------+
|              105|MOBILE_PHONE_PLAT...|            ApplePay|                09/25|
+-----------------+--------------------+--------------------+---------------------+
only showing top 1 row



In [36]:
user_play_session_df.show(2)

+---------------+-------+--------------------+--------------------+------------+-----------+-----------+
|play_session_id|user_id|      start_datetime|        end_datetime|channel_code|status_code|total_score|
+---------------+-------+--------------------+--------------------+------------+-----------+-----------+
|              0|    309|2024-12-11 13:26:...|2024-12-11 18:01:...|     BROWSER|  COMPLETED|       1081|
|              1|    179|2024-02-13 16:51:...|2024-02-13 23:17:...|      MOBILE|    ABORTED|       4271|
+---------------+-------+--------------------+--------------------+------------+-----------+-----------+
only showing top 2 rows



In [37]:
dim_user_play_session=user_play_session_df.dropDuplicates()

In [38]:
dim_payment_detail=user_payment_detail_df.dropDuplicates()

In [39]:
dim_payment_detail.show(2)

+-----------------+--------------------+--------------------+---------------------+
|payment_detail_id| payment_method_code|payment_method_value|payment_method_expiry|
+-----------------+--------------------+--------------------+---------------------+
|              105|MOBILE_PHONE_PLAT...|            ApplePay|                09/25|
|              116|MOBILE_PHONE_PLAT...|          SamsungPay|                03/27|
+-----------------+--------------------+--------------------+---------------------+
only showing top 2 rows



In [40]:
dim_registration=user_registration_df.dropDuplicates()

In [41]:
dim_registration.show(2)

+--------------------+-------+--------+-------------------+----------+---------+
|user_registration_id|user_id|username|              email|first_name|last_name|
+--------------------+-------+--------+-------------------+----------+---------+
|                 103|    246|  lcantu| lcantu@example.net|  Lorraine|    Cantu|
|                 144|    359| bbarnes|bbarnes@example.net|     Brett|   Barnes|
+--------------------+-------+--------+-------------------+----------+---------+
only showing top 2 rows



In [102]:
dim_user_plan=user_plan_df.dropDuplicates()

In [103]:
dim_user_plan.show(2)

+--------------------+-----------------+-------+-------------------+-------------------+
|user_registration_id|payment_detail_id|plan_id|         start_date|           end_date|
+--------------------+-----------------+-------+-------------------+-------------------+
|                   9|                9|      2|2024-02-14 01:00:00|2024-12-16 01:00:00|
|                 108|              108|      1|2024-10-16 02:00:00|9999-01-01 01:00:00|
+--------------------+-----------------+-------+-------------------+-------------------+
only showing top 2 rows



In [ ]:
# Create Fact table based on user session 

In [55]:
fact_play_sessions_df = dim_user_play_session.alias("ups") \
    .join(dim_user.alias("u"), on="user_id", how="inner") \
    .join(dim_channel.alias("c"), dim_user_play_session.channel_code == dim_channel.play_session_channel_code, "inner") \
    .join(dim_status.alias("s"), dim_user_play_session.status_code == dim_status.play_session_status_code, "inner") \
    .select(
        col("ups.play_session_id"),
        col("ups.user_id"),
        col("u.ip_address"),
        col("u.social_media_handle"),
        col("u.email"),
        col("c.play_session_channel_code").alias("channel_code"),
        col("c.english_description").alias("channel_name"),
        col("s.play_session_status_code").alias("status_code"),
        col("s.english_description").alias("status_name"),
        col("ups.start_datetime"),
        col("ups.end_datetime"),
        col("ups.total_score")
    )

In [162]:
fact_play_sessions_df.show(1)

+---------------+-------+-------------+-------------------+------------------+------------+------------------+-----------+-----------------+--------------------+--------------------+-----------+------------------------+
|play_session_id|user_id|   ip_address|social_media_handle|             email|channel_code|      channel_name|status_code|      status_name|      start_datetime|        end_datetime|total_score|session_duration_minutes|
+---------------+-------+-------------+-------------------+------------------+------------+------------------+-----------+-----------------+--------------------+--------------------+-----------+------------------------+
|            118|    418|10.162.149.64|             ccross|ccross@example.org|      MOBILE|Mobile application|  COMPLETED|Session completed|2024-06-26 12:33:...|2024-06-26 13:37:...|        772|                    64.0|
+---------------+-------+-------------+-------------------+------------------+------------+------------------+----------

In [ ]:
#To know which channel are using more

In [62]:
fact_play_sessions_df.groupby("channel_code").count().show()

+------------+-----+
|channel_code|count|
+------------+-----+
|      MOBILE|  930|
|     BROWSER|  941|
+------------+-----+



In [ ]:
# Find the play count 

In [163]:
fact_play_sessions_df \
    .groupBy("channel_code") \
    .agg(
        F.count("*").alias("play_count"),
        F.sum("total_score").alias("total_score_sum")
    ) \
    .orderBy("channel_code") \
    .show()

+------------+----------+---------------+
|channel_code|play_count|total_score_sum|
+------------+----------+---------------+
|     BROWSER|       941|        2434675|
|      MOBILE|       930|        2321364|
+------------+----------+---------------+



In [ ]:
#To know the playing status details

In [89]:
fact_play_sessions_df.groupby("status_code").count().orderBy(col("count").desc()).show(2)

+-----------+-----+
|status_code|count|
+-----------+-----+
|    ABORTED|  661|
|  COMPLETED|  607|
+-----------+-----+
only showing top 2 rows



In [ ]:
#To find the duration of play -- 

In [ ]:
from pyspark.sql.functions import col, unix_timestamp, round

In [81]:
fact_play_sessions_df.withColumn(
    "session_duration_minutes",
    round((unix_timestamp("end_datetime") - unix_timestamp("start_datetime")) / 60, 2)
).select("play_session_id","user_id","channel_code","total_score","session_duration_minutes").show(2)
#fact_play_sessions_df.
#.withColumn("session_duration_minutes", round((unix_timestamp("end_datetime") - unix_timestamp("start_datetime")) / 60, 2))


+---------------+-------+------------+-----------+------------------------+
|play_session_id|user_id|channel_code|total_score|session_duration_minutes|
+---------------+-------+------------+-----------+------------------------+
|            118|    418|      MOBILE|        772|                    64.0|
|            161|     73|      MOBILE|       2054|                   201.0|
+---------------+-------+------------+-----------+------------------------+
only showing top 2 rows



In [90]:
#fact_play_sessions_df.show(2)

In [151]:
#fact_play_sessions_df.write.mode("overwrite").parquet("C:/Users/lenovo/Downloads/fact_play_sessions")

In [ ]:
# Fact based on user plan and payment

In [98]:
dim_user.show(2)

+-------+--------------+-------------------+-----+
|user_id|    ip_address|social_media_handle|email|
+-------+--------------+-------------------+-----+
|     41|10.130.164.103|               jkim| null|
|    103|  172.19.40.11|           tshannon| null|
+-------+--------------+-------------------+-----+
only showing top 2 rows



In [104]:
dim_user_plan.show(2)

+--------------------+-----------------+-------+-------------------+-------------------+
|user_registration_id|payment_detail_id|plan_id|         start_date|           end_date|
+--------------------+-----------------+-------+-------------------+-------------------+
|                   9|                9|      2|2024-02-14 01:00:00|2024-12-16 01:00:00|
|                 108|              108|      1|2024-10-16 02:00:00|9999-01-01 01:00:00|
+--------------------+-----------------+-------+-------------------+-------------------+
only showing top 2 rows



In [107]:
fact_user_plan_df = (
    dim_user_plan
    # Join with user registration
    .join(dim_registration, "user_registration_id", "left")
    # Join with user dimension
    .join(dim_user, "user_id", "left")
    # Join with plan dimension
    .join(dim_plan, "plan_id", "left")
    # Join with payment frequency dimension
    .join(dim_payment_frequency, "payment_frequency_code", "left")
    # Join with payment details
    .join(dim_payment_detail, "payment_detail_id", "left"))

In [161]:
fact_user_plan_df.describe()

DataFrame[summary: string, payment_detail_id: string, payment_frequency_code: string, plan_id: string, user_id: string, user_registration_id: string, username: string, email: string, first_name: string, last_name: string, ip_address: string, social_media_handle: string, email: string, cost_amount: string, english_description: string, french_description: string, payment_method_code: string, payment_method_value: string, payment_method_expiry: string]

In [77]:
fact_user_plan_df = (
    dim_user_plan
    # Join with user registration
    .join(dim_registration, "user_registration_id", "left")
    # Join with user dimension
    .join(dim_user, "user_id", "left")
    # Join with plan dimension
    .join(dim_plan, "plan_id", "left")
    # Join with payment frequency dimension
    .join(dim_payment_frequency, "payment_frequency_code", "left")
    # Join with payment details
    .join(dim_payment_detail, "payment_detail_id", "left")
    # Select fact table columns
    .select(
        col("user_registration_id"),
        col("user_id"),
        col("username"),
        col("first_name"),
        col("last_name"),
        col("user_df.email").alias("user_email"),
        col("ip_address"),
        col("social_media_handle"),
        col("plan_id"),
        col("payment_frequency_code"),
        col("english_description").alias("payment_frequency_english"),
        col("cost_amount"),
        col("payment_method_code"),
        col("payment_method_value"),
        col("payment_method_expiry"),
        col("start_date"),
        col("end_date")
    )
)


In [ ]:
# To find the highest paid user

In [115]:
fact_user_plan_df \
    .groupBy("user_id") \
    .agg(F.sum("cost_amount").alias("total_cost")) \
    .orderBy(F.desc("total_cost")) \
    .show(10)

+-------+----------+
|user_id|total_cost|
+-------+----------+
|    211|     24.98|
|    204|     24.98|
|    456|     24.98|
|      7|     24.98|
|     19|     24.98|
|    244|     24.98|
|    486|     16.98|
|     78|     16.98|
|    367|     16.98|
|    375|     16.98|
+-------+----------+
only showing top 10 rows



In [ ]:
# Find most expensive plan

In [116]:
fact_user_plan_df.select("user_id", "plan_id", "cost_amount") \
    .orderBy(F.desc("cost_amount")).show(5)

+-------+-------+-----------+
|user_id|plan_id|cost_amount|
+-------+-------+-----------+
|    395|      3|      14.99|
|     78|      3|      14.99|
|    375|      3|      14.99|
|    461|      3|      14.99|
|    166|      3|      14.99|
+-------+-------+-----------+
only showing top 5 rows



In [ ]:
# Find the users based on total cost-- rank 

In [154]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

In [155]:
f1=fact_user_plan_df \
    .groupBy("user_id") \
    .agg(F.sum("cost_amount").alias("total_spend"))

In [119]:
f1.

+-------+-----------+
|user_id|total_spend|
+-------+-----------+
|    148|       9.99|
|    496|      14.99|
|     31|       9.99|
|     85|       1.99|
+-------+-----------+
only showing top 4 rows



In [136]:
window_spec = Window.orderBy(F.desc("total_spend"))

In [140]:
ranked_users_df = f1.withColumn("rank", F.rank().over(window_spec)) \
                    .orderBy(F.asc("rank"))

In [141]:
ranked_users_df.show(5)

+-------+-----------+----+
|user_id|total_spend|rank|
+-------+-----------+----+
|    211|      24.98|   1|
|     19|      24.98|   1|
|    456|      24.98|   1|
|      7|      24.98|   1|
|    244|      24.98|   1|
+-------+-----------+----+
only showing top 5 rows



In [143]:
ranked_users_df1 = f1.withColumn("rank", F.dense_rank().over(window_spec)) \
                    .orderBy(F.asc("rank"))

In [145]:
ranked_users_df1.show(10)

+-------+-----------+----+
|user_id|total_spend|rank|
+-------+-----------+----+
|    211|      24.98|   1|
|    204|      24.98|   1|
|    456|      24.98|   1|
|      7|      24.98|   1|
|     19|      24.98|   1|
|    244|      24.98|   1|
|     78|      16.98|   2|
|    375|      16.98|   2|
|    367|      16.98|   2|
|    259|      16.98|   2|
+-------+-----------+----+
only showing top 10 rows



In [152]:
# Save fact table


In [153]:
#fact_user_plan_df.write.mode("overwrite").parquet("C:/Users/lenovo/Downloads/fact_play_sessions")

In [ ]:
#subscription details

In [157]:
fact_user_plan_df.groupby("payment_frequency_code").count().show()

+----------------------+-----+
|payment_frequency_code|count|
+----------------------+-----+
|               MONTHLY|   74|
|              ANNUALLY|   66|
|               ONETIME|   75|
+----------------------+-----+



In [ ]:
Revenue generated through app vs mobile

In [ ]:
fact_user_plan_df

In [167]:
fact_final_df = (
    fact_user_plan_df
    # Join with user registration
    .join(fact_play_sessions_df, "user_id", "inner"))

In [170]:
fact_final_df \
    .groupBy("channel_code") \
    .agg(
        F.count("*").alias("play_count"),
        F.sum("cost_amount").alias("total_revenue")
    ) \
    .orderBy("channel_code") \
    .show()

+------------+----------+------------------+
|channel_code|play_count|     total_revenue|
+------------+----------+------------------+
|     BROWSER|       425| 3689.749999999962|
|      MOBILE|       399|3514.0099999999647|
+------------+----------+------------------+



In [168]:
fact_final_df.show(1)

+-------+-----------------+----------------------+-------+--------------------+-------------------+-------------------+--------+------------------+----------+---------+--------------+-------------------+------------------+-----------+-------------------+------------------+--------------------+--------------------+---------------------+---------------+--------------+-------------------+------------------+------------+------------+-----------+-----------------+--------------------+--------------------+-----------+------------------------+
|user_id|payment_detail_id|payment_frequency_code|plan_id|user_registration_id|         start_date|           end_date|username|             email|first_name|last_name|    ip_address|social_media_handle|             email|cost_amount|english_description|french_description| payment_method_code|payment_method_value|payment_method_expiry|play_session_id|    ip_address|social_media_handle|             email|channel_code|channel_name|status_code|      status

In [164]:
#fact_user_plan_df.show(1)

+-----------------+----------------------+-------+-------+--------------------+-------------------+-------------------+--------+------------------+----------+---------+--------------+-------------------+------------------+-----------+-------------------+------------------+--------------------+--------------------+---------------------+
|payment_detail_id|payment_frequency_code|plan_id|user_id|user_registration_id|         start_date|           end_date|username|             email|first_name|last_name|    ip_address|social_media_handle|             email|cost_amount|english_description|french_description| payment_method_code|payment_method_value|payment_method_expiry|
+-----------------+----------------------+-------+-------+--------------------+-------------------+-------------------+--------+------------------+----------+---------+--------------+-------------------+------------------+-----------+-------------------+------------------+--------------------+--------------------+---------

In [165]:
#fact_play_sessions_df.show(1)

+---------------+-------+-------------+-------------------+------------------+------------+------------------+-----------+-----------------+--------------------+--------------------+-----------+------------------------+
|play_session_id|user_id|   ip_address|social_media_handle|             email|channel_code|      channel_name|status_code|      status_name|      start_datetime|        end_datetime|total_score|session_duration_minutes|
+---------------+-------+-------------+-------------------+------------------+------------+------------------+-----------+-----------------+--------------------+--------------------+-----------+------------------------+
|            118|    418|10.162.149.64|             ccross|ccross@example.org|      MOBILE|Mobile application|  COMPLETED|Session completed|2024-06-26 12:33:...|2024-06-26 13:37:...|        772|                    64.0|
+---------------+-------+-------------+-------------------+------------------+------------+------------------+----------

In [ ]:
fact_play_sessions_df \
    .groupBy("channel_code") \
    .agg(
        F.count("*").alias("play_count"),
        F.sum("total_score").alias("total_score_sum")
    ) \
    .orderBy("channel_code") \
    .show()